In [2]:
import pandas as pd
import unicodedata
import sys
import re
import io
import random
sys.path.append('../transformer-pytorch/')

## Data Shuffle

In [58]:
deu_eng_file = './deu.txt'
deu_eng_file_shuffle = './deu_shuffle.txt'


In [59]:
def shuffle_file(path, new_path):
    with open(path,'r') as source:
        data = [(random.random(), line) for line in source]
        data.sort()
    with open(new_path,'w') as target:
        for _, line in data:
            target.write(line)

In [60]:
shuffle_file(deu_eng_file, deu_eng_file_shuffle)

## Data Preprocessing

1. 각 문장에 start와 end 토큰을 추가
2. 특정 문자를 제거함으로써 문장을 정리
3. 단어 인덱스와 아이디(ID) 인덱스를 생성 (단어 → 아이디(ID), 아이디(ID) → 단어로 매핑된 딕셔너리).
4. 각 문장을 입력층의 최대 길이만큼 패딩(padding)을 수행

In [12]:
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  w = '<start> ' + w + ' <end>'
  return w

In [13]:
en_sentence = u"When you're done, close the door."
de_sentence = u"Wenn du fertig bist, schließ die Tür!"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(de_sentence).encode('utf-8'))

<start> when you re done , close the door . <end>
b'<start> wenn du fertig bist , schlie die tur ! <end>'


In [44]:
# 1. 문장에 있는 억양을 제거합니다.
# 2. 불필요한 문자를 제거하여 문장을 정리합니다.
# 3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [46]:
en, de, _ = create_dataset(deu_eng_file, 30000)

## Load original dataset

In [49]:
# 1. 문장에 있는 억양을 제거합니다.
# 2. 불필요한 문자를 제거하여 문장을 정리합니다.
# 3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]
def load_origin(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)


In [65]:
en_ori, de_ori, _ = load_origin(deu_eng_file_shuffle, 30000)

### Test Data file 생성

In [66]:
with open('testset.txt', 'w') as target:
    for i in range(len(en_ori)):
        data = en_ori[i] + '\t' + de_ori[i] + '\n'
        target.write(data)
